In [ ]:
#!pip install numpy scipy

In [8]:
import numpy as np
from scipy.stats import norm
import copy

class Option:
    def __init__(self, 
                 S0 : float, 
                 K : float, 
                 r : float,
                 sigma : float,
                 T : float, 
                 is_call : bool,
                 is_american : bool):
        self.S0 = S0
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T
        self.is_call = is_call
        self.is_american = is_american
    
    def payoff(self, S):
        mult = 1 if self.is_call else -1
        return max(mult * (S - self.K), 0.0)
    
    def discount(self, dt):
        return np.exp(-self.r * dt)

class OptionPricer:
    def price(self, inst : Option) -> float:
        raise NotImplementedError
    
class BlackSholesPricer(OptionPricer):
        
    def price(self, inst : Option) -> float:
        if inst.is_american:
            raise RuntimeError("BlackSholesPricer does not support American Option")

        d1 = (np.log(inst.S0 / inst.K) + (inst.r + (inst.sigma * inst.sigma) / 2) * inst.T) / (inst.sigma * np.sqrt(inst.T))
        d2 = d1 - inst.sigma * np.sqrt(inst.T)

        call = norm.cdf(d1) * inst.S0 - norm.cdf(d2) * inst.K * inst.discount(inst.T)
        if inst.is_call:
            return call
        else:
            #deal with put American Option
            return call - inst.S0 + inst.K * inst.discount(inst.T)

        
class BinomialTreePricer(OptionPricer):
    def __init__(self, N_per_year : int = 252) -> None:
        self.N_per_year = N_per_year

    def price(self, inst : Option) -> float:

        N = int(self.N_per_year * inst.T)
        dt = 1.0 / N
        u = np.exp(inst.r * dt + inst.sigma * np.sqrt(dt))
        d = np.exp(inst.r * dt - inst.sigma * np.sqrt(dt))
        p = (np.exp(inst.r * dt) - d) / (u - d)
        df = inst.discount(dt)

        SGrid = [np.zeros(i+1) for i in range(N+1)]
        VGrid = [-1 * np.ones(i+1) for i in range(N+1)] # -1 for not evaluated

        def value_recursive(i, j, S):
            SGrid[i][j] = S
            if VGrid[i][j] != -1:
                return VGrid[i][j] # already evaluated, return directly
            if i == N:
                # reach T, calculate the final payoff
                VGrid[i][j] = inst.payoff(S)
                return VGrid[i][j]
            
            holding_value = df * (p * value_recursive(i+1, j+1, S * u) + \
                                  (1.0 - p) * value_recursive(i+1, j, S * d))
            if not inst.is_american:
                VGrid[i][j] = holding_value
            else:
                exercise_value = inst.payoff(S)
                VGrid[i][j] = max(holding_value, exercise_value)
            return VGrid[i][j]
        
        value_recursive(0, 0, inst.S0)
        return VGrid[0][0]

In [11]:
import time
start_time = time.time()
inst_eo = Option(100, 100, 0.01, 0.30, 1.0, False, False)
inst_ao = Option(100, 100, 0.01, 0.30, 1.0, False, True)

bs_pricer = BlackSholesPricer()
print("Black Scholes Model for European Option:",bs_pricer.price(inst_eo))

tree_pricer = BinomialTreePricer()
print("Binomial Tree Model for European Option:" ,tree_pricer.price(inst_eo))
print("Binomial Tree Model for American Option:" ,tree_pricer.price(inst_ao))



print("--- %s seconds ---" % (time.time() - start_time))

        


Black Scholes Model for European Option: 11.37325083870087
Binomial Tree Model for European Option: 11.379647283888838
Binomial Tree Model for American Option: 11.454427617920302
--- 0.2158823013305664 seconds ---


$$ \frac{\partial  V(t,x)}{\partial t} = 
+ \frac{-\sigma^2}{2}  \frac{\partial^2  V(t,x)}{\partial x^2}
+ ( \frac{1}{2}\sigma^2 - r) \frac{\partial V(t,x)}{\partial x} 
+ r  V(t,x) $$


In [4]:
class FiniteDifferencePricer(OptionPricer):

    def __init__(self, N_per_year : int = 252, N_logspace_grid = 500) -> None:
        self.N_per_year = N_per_year
        self.N_logspace_grid = N_logspace_grid // 2 * 2 # make sure even

    
    def build_time_grid(self) -> None:
        self.N = int(self.N_per_year * self.inst.T)
        self.dt = self.inst.T / self.N

    def build_space_grid(self) -> bool:
        inst = self.inst
        self.x0 = np.log(inst.S0)
        k = np.log(inst.K)
        #influence region around the spot
        xspan = 5 * inst.sigma * np.sqrt(inst.T)
        dx = (2 * xspan) / self.N_logspace_grid
        xlow = self.x0 - xspan
        xgrid = [xlow + i * dx for i in range(self.N_logspace_grid + 1)]
        #the strike is outside the probable range, price it as european later
        if k < xgrid[0] or k > xgrid[-1]:
            return False

        ddx = xgrid[np.searchsorted(xgrid, k)] - k

        self.xgrid = [x - ddx for x in xgrid]
        self.dx = dx
        self.M = len(self.xgrid)

        return True
    
    # luckily we are dealing with constant vol and constant r
    def build_coefficients(self) -> None:
        inst = self.inst
        sigma2 = inst.sigma * inst.sigma
        F =  inst.r - 0.5 * sigma2

        dt = self.dt
        dx = self.dx
        
        t1 = dt / (4 * dx * dx)
        t2 = dt / (4 * dx)

        self.a = -t1 * sigma2 - t2 * F
        self.b = 1 + 2 * t1 * sigma2 + 0.5 * dt * inst.r
        self.c = -t1 * sigma2 + t2 * F

    def value_on_grid(self):
        inst = self.inst
        a, b, c = self.a, self.b, self.c

        E = np.zeros(self.M) # exercise value
        U = np.zeros(self.M)
        Utilde = np.zeros(self.M)

        # initiate the payoff for t = T
        for i in range(self.M):
            x = self.xgrid[i]
            E[i] = inst.payoff(np.exp(x))
            U[i] = inst.payoff(np.exp(x))
            
        # for put
        #Thomas Algorithm
        def backward_substitution():
            # helpers for substitution
            z = np.zeros(self.M)
            y = np.zeros(self.M)
            z[-1] = 1
            #recall that M is the length of M
            j = self.M - 1
            while j > 0:
                z[j-1] = -c / (a * z[j] + b)
                y[j-1] = (U[j] - a * y[j]) / (a * z[j] + b)

                if inst.is_american and (E[j] - E[j-1] - y[j-1]) / (z[j-1] - 1) < E[j-1]:
                    # check the boundary condition and set the remaining to exercise value
                    Utilde[:(j+1)] = E[:(j+1)]
                    break

                j = j - 1
            
            Utilde[0] = E[0]
            while j < self.M - 1:
                Utilde[j+1] = z[j] * Utilde[j] + y[j]
                j = j + 1
            
        def forward_substitution():
            # helpers for substitution
            z = np.zeros(self.M)
            y = np.zeros(self.M)
            z[0] = 1

            j = 0
            while j < self.M - 1:
                z[j+1] = -a / (c * z[j] + b)
                y[j+1] = (U[j] - c * y[j]) / (c * z[j] + b)

                if inst.is_american and (E[j+1] - E[j] + y[j+1]) / (1 - z[j+1]) < E[j+1]:
                    # check the boundary condition and set the remaining to exercise value
                    Utilde[j:] = E[j:]
                    break

                j = j + 1
            
            Utilde[-1] = E[-1]
            while j > 0:
                Utilde[j-1] = z[j] * Utilde[j] + y[j]
                

        for i in range(self.N, 0, -1):
            if inst.is_call:
                forward_substitution()
            else:
                backward_substitution()
            U = 2 * Utilde - U
            #print(i , U, Utilde)

        return U

    def price(self, inst : Option) -> float:

        self.inst = inst
        inst_eo = copy.deepcopy(inst)
        
        if not self.build_space_grid():
            # the strike is outside the probable range, price it as european
            inst_eo = copy.deepcopy(inst)
            inst_eo.is_american = False
            pricer = BlackSholesPricer()
            return pricer.price(inst_eo)
        
        self.build_time_grid()
        self.build_coefficients()

        U = self.value_on_grid()
        from scipy.interpolate import Akima1DInterpolator
        interp = Akima1DInterpolator(self.xgrid, U)
        
        return float(interp(self.x0))



In [12]:
fd_pricer = FiniteDifferencePricer(500)
tree_pricer = BinomialTreePricer(500)

inst_ao = Option(100, 100, 0.01, 0.30, 1.0, False, True)
print(fd_pricer.price(inst_ao), tree_pricer.price(inst_ao))

inst_ao = Option(100, 110, 0.02, 0.80, 1.0, False, True)
print(fd_pricer.price(inst_ao), tree_pricer.price(inst_ao))


11.446557631588316 11.45272955105887
36.60259691645816 36.59744834376177


In [13]:
print("-----end------")

-----end------
